### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
#importar as bibliotecas
import numpy as np
import pandas as pd #ler a base de dados
from pathlib import Path #interage com pasta e arquivos do computador

# para o e-mail 
import smtplib #enviar e-mail
from email.mime.multipart import MIMEMultipart #cria a instância do e-mail
from email.mime.text import MIMEText #adicionar o texto do e-mail
from email.mime.base import MIMEBase #adicionar anexo
from email.encoders import encode_base64 #converter anexo em base 64 que é o padrão do e-mail


In [2]:
#importando a base de dados
vendas_df = pd.read_excel('Bases de Dados/Vendas.xlsx')
email_df = pd.read_excel('Bases de Dados/Emails.xlsx')
lojas_df = pd.read_csv('Bases de Dados/Lojas.csv', encoding='latin1', sep=';')

In [3]:
display(vendas_df)
email_df['E-mail'] = email_df['E-mail'].apply(lambda x: x.replace('pythonimpressionador', 'sapientiapython'))
display(email_df)
display(lojas_df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204
100995,69996,2019-12-26,17,Mochila,4,270,1080
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108


,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,sapientiapython+helena@gmail.com
1,Shopping Midway Mall,Alice,sapientiapython+alice@gmail.com
2,Norte Shopping,Laura,sapientiapython+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,sapientiapython+manuela@gmail.com
4,Shopping União de Osasco,Valentina,sapientiapython+valentina@gmail.com
5,Shopping Center Interlagos,Sophia,sapientiapython+sophia@gmail.com
6,Rio Mar Recife,Isabella,sapientiapython+isabella@gmail.com
7,Salvador Shopping,Heloisa,sapientiapython+heloisa@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,sapientiapython+luiza@gmail.com
9,Shopping Center Leste Aricanduva,Julia,sapientiapython+julia@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco
5,6,Shopping Center Interlagos
6,7,Rio Mar Recife
7,8,Salvador Shopping
8,9,Rio Mar Shopping Fortaleza
9,10,Shopping Center Leste Aricanduva


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [4]:
#dia do indicador é o dia mais recente, portanto, é a maior data
dia_indicador = vendas_df['Data'].max()
dia_indicador

#juntar os df de vendas e lojas
juntos_df = vendas_df.merge(lojas_df, on='ID Loja')
juntos_df

#adicionar todas as tabelas no dicionário separada pelas chaves da loja
dicionario_lojas = {} 

for loja in lojas_df['Loja']:
    dicionario_lojas[loja] = juntos_df.loc[juntos_df['Loja']==loja]



### Passo 3 - Salvar a planilha na pasta de backup

In [5]:
nome_backup = Path('Backup Arquivos Lojas')
for loja in lojas_df['Loja']:
    #criar a pasta do backup
    caminho = nome_backup / loja
    if not caminho.exists():
        caminho.mkdir(parents=True)
    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day} {loja}.xlsx'   
    dicionario_lojas[loja].to_excel(caminho/nome_arquivo)
        
    
    

### Passo 4 - Calcular o indicador para 1 loja

In [6]:
# Faturamento -> Meta Ano: 1.650.000 / Meta Dia: 1000
# Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Ano: 120 / Meta Dia: 4
# Ticket Médio por Venda -> Meta Ano: 500 / Meta Dia: 500

meta_faturamento_ano = 1650000
meta_faturamento_dia = 1000

meta_diversidade_produtos_ano = 120
meta_diversidade_produtos_dia = 4

meta_ticket_medio_ano = 500
meta_ticket_medio_dia = 500

#cálculo das metas
vendas_loja = dicionario_lojas['Shopping Morumbi']
vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador]

#faturamento
faturamento_ano = vendas_loja['Valor Final'].sum()
faturamento_dia = vendas_loja_dia['Valor Final'].sum()
print(faturamento_ano)
print(faturamento_dia)


#diversidade de produtos
diversidade_produtos_ano = len(vendas_loja['Produto'].unique())
diversidade_produtos_dia = len(vendas_loja_dia['Produto'].unique())
print(diversidade_produtos_ano)
print(diversidade_produtos_dia)

#ticket médio por vendas
tabela_agrupada = vendas_loja.groupby('Código Venda')[['Código Venda', 'Valor Final']].sum()
ticket_medio_ano = round(tabela_agrupada['Valor Final'].mean(), 2)


tabela_agrupada_dia = vendas_loja_dia.groupby('Código Venda')[['Código Venda', 'Valor Final']].sum()
media = tabela_agrupada_dia['Valor Final'].mean()
if np.isnan(media):
    media = 0
ticket_medio_dia = media
print(ticket_medio_ano)
print(ticket_medio_dia)

1586444
0
120
0
765.66
0


In [7]:
#criando a função de cor
def cor(meta, resultado):
    valor = resultado - meta
    if valor >= 0:
        cor = 'green'
    else:
        cor = 'red'
    return cor

### Passo 5 - Enviar por e-mail para o gerente

Caso opte por enviar pelo hotmail utilize as seguintes configurações:
- host = 'smtp.office365.com'
- port = '587'
- usuario = email do hotmail
- senha = senha do hotmail

In [8]:
#criando o email
# 1 - inicializando as variáveis
host = 'smtp.gmail.com' #servidor do g-mail
port = '587'
usuario = 'seu email' #email que vai enviar a mensagem
senha = 'senha gerada pelo app do gmail' #senha gerada na parte de segurança

# 2 - criando o email
mensagem = MIMEMultipart()
mensagem['From'] = usuario
mensagem['Subject'] = f'One page do dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
mensagem['To'] = email_df['E-mail'][0]
texto = f"""
<p>Bom dia, {email_df['Gerente'][0]} </p>
<p> O resultado de ontem <b>{dia_indicador.day}/{dia_indicador.month}</b> da <b>Loja {loja}</b></p>

<table>
  <tr>
    <th>Indicadore</th>
    <th>Valor Dia</th>
    <th>Meta Dia</th>
    <th>Cenário Dia</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td style='text-align: center'>{faturamento_dia}</td>
    <td style='text-align: center'>{meta_faturamento_dia}</td>
    <td style='text-align: center'><font color = '{cor(meta_faturamento_dia, faturamento_dia)}'>◙</font></td>
  </tr>
  <tr>
    <td>Diversidade de Produtos</td>
    <td style='text-align: center'>{diversidade_produtos_dia}</td>
    <td style='text-align: center'>{meta_diversidade_produtos_dia}</td>
    <td style='text-align: center'><font color = '{cor(meta_diversidade_produtos_dia, diversidade_produtos_dia)}'>◙</font></td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td style='text-align: center'>{meta_ticket_medio_dia}</td>
    <td style='text-align: center'>{ticket_medio_dia}</td>
    <td style='text-align: center'><font color = '{cor(meta_ticket_medio_dia, ticket_medio_dia)}'>◙</font></td>
  </tr>
  
</table>
<br>
<table>
  <tr>
    <th>Indicadore</th>
    <th>Valor Dia</th>
    <th>Meta Dia</th>
    <th>Cenário Dia</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td style='text-align: center'>{faturamento_ano}</td>
    <td style='text-align: center'>{meta_faturamento_ano}</td>
    <td style='text-align: center'><font color = '{cor(meta_faturamento_ano, faturamento_ano)}'>◙</font></td>
  </tr>
  <tr>
    <td>Diversidade de Produtos</td>
    <td style='text-align: center'>{diversidade_produtos_ano}</td>
    <td style='text-align: center'>{meta_diversidade_produtos_ano}</td>
    <td style='text-align: center'><font color = '{cor(meta_diversidade_produtos_ano, diversidade_produtos_ano)}'>◙</font></td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td style='text-align: center'>{meta_ticket_medio_ano}</td>
    <td style='text-align: center'>{ticket_medio_ano}</td>
    <td style='text-align: center'><font color = '{cor(meta_ticket_medio_ano, ticket_medio_ano)}'>◙</font></td>
  </tr>
  
</table>


<p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att.,Victor</p>

"""
mensagem.attach(MIMEText(texto, 'html'))
#adicionando um anexo
loja = email_df['Loja'][0]
caminho = Path.cwd() / nome_backup / loja / f'{dia_indicador.month}_{dia_indicador.day} {loja}.xlsx'
with open(caminho, 'rb') as  arquivo:
    anexo = MIMEBase('apllication', 'octet-stream') # 'apllication', 'octet-stream' é usado para dados binários arbitrários. Salvar o dado
    anexo.set_payload(arquivo.read()) #adicionando o arquivo binário
    encode_base64(anexo) #codificando o arquivo na base 64, ASCII
    anexo.add_header('Content-Disposition', 'attachment', filename = f'{dia_indicador.month}_{dia_indicador.day} {loja}.xlsx') #criando o cabeçalho que vai ser adicionando à parte ao anexo
    mensagem.attach(anexo) #adicionando no corpo do e-mail






with smtplib.SMTP(host, port) as servidor:
    servidor.starttls() #protocolo de segurança
    servidor.login(usuario, senha)
    servidor.sendmail(mensagem['From'], mensagem['To'], mensagem.as_string().encode('utf-8'))

### Passo 6 - Automatizar todas as lojas

In [ ]:
host = 'smtp.gmail.com' #servidor do g-mail
port = '587'
usuario = 'seu email' #email que vai enviar a mensagem
senha = 'senha gerada pelo app do gmail' #senha gerada na parte de segurança



meta_faturamento_ano = 1650000
meta_faturamento_dia = 1000

meta_diversidade_produtos_ano = 120
meta_diversidade_produtos_dia = 4

meta_ticket_medio_ano = 500
meta_ticket_medio_dia = 500

for indice in email_df.index:
    if email_df.loc[indice, 'Gerente'] != 'Diretoria':
        loja = email_df.loc[indice, 'Loja'] #nome da loja
        gerente = email_df.loc[indice, 'Gerente'] #nome do gerente
        email = email_df.loc[indice, 'E-mail'] #mail
        #cálculo das metas
        vendas_loja = dicionario_lojas[loja]
        vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador]

        #faturamento
        faturamento_ano = vendas_loja['Valor Final'].sum()
        faturamento_dia = vendas_loja_dia['Valor Final'].sum()
#         print(faturamento_ano)
#         print(faturamento_dia)


        #diversidade de produtos
        diversidade_produtos_ano = len(vendas_loja['Produto'].unique())
        diversidade_produtos_dia = len(vendas_loja_dia['Produto'].unique())
#         print(diversidade_produtos_ano)
#         print(diversidade_produtos_dia)

        #ticket médio por vendo
        tabela_agrupada = vendas_loja.groupby('Código Venda')[['Código Venda', 'Valor Final']].sum()
        ticket_medio_ano = tabela_agrupada['Valor Final'].mean()
        tabela_agrupada_dia = vendas_loja_dia.groupby('Código Venda')[['Código Venda', 'Valor Final']].sum()
        ticket_medio_dia = tabela_agrupada_dia['Valor Final'].mean()
#         print(ticket_medio_ano)
#         print(ticket_medio_dia)



        # 2 - criando o email
        mensagem = MIMEMultipart()
        mensagem['From'] = usuario
        mensagem['Subject'] = f'One page do dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
        mensagem['To'] = email
        texto = f"""
        <p>Bom dia, {gerente} </p>
        <p> O resultado de ontem <b>{dia_indicador.day}/{dia_indicador.month}</b> da <b>Loja {loja}</b></p>

        <table>
          <tr>
            <th>Indicadore</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
          </tr>
          <tr>
            <td>Faturamento</td>
            <td style='text-align: center'>{faturamento_dia}</td>
            <td style='text-align: center'>{meta_faturamento_dia}</td>
            <td style='text-align: center'><font color = '{cor(meta_faturamento_dia, faturamento_dia)}'>◙</font></td>
          </tr>
          <tr>
            <td>Diversidade de Produtos</td>
            <td style='text-align: center'>{diversidade_produtos_dia}</td>
            <td style='text-align: center'>{meta_diversidade_produtos_dia}</td>
            <td style='text-align: center'><font color = '{cor(meta_diversidade_produtos_dia, diversidade_produtos_dia)}'>◙</font></td>
          </tr>
          <tr>
            <td>Ticket Médio</td>
            <td style='text-align: center'>{meta_ticket_medio_dia}</td>
            <td style='text-align: center'>{ticket_medio_dia}</td>
            <td style='text-align: center'><font color = '{cor(meta_ticket_medio_dia, ticket_medio_dia)}'>◙</font></td>
          </tr>

        </table>
        
        <br>
        <table>
          <tr>
            <th>Indicadore</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
          </tr>
          <tr>
            <td>Faturamento</td>
            <td style='text-align: center'>{faturamento_ano}</td>
            <td style='text-align: center'>{meta_faturamento_ano}</td>
            <td style='text-align: center'><font color = '{cor(meta_faturamento_ano, faturamento_ano)}'>◙</font></td>
          </tr>
          <tr>
            <td>Diversidade de Produtos</td>
            <td style='text-align: center'>{diversidade_produtos_ano}</td>
            <td style='text-align: center'>{meta_diversidade_produtos_ano}</td>
            <td style='text-align: center'><font color = '{cor(meta_diversidade_produtos_ano, diversidade_produtos_ano)}'>◙</font></td>
          </tr>
          <tr>
            <td>Ticket Médio</td>
            <td style='text-align: center'>{meta_ticket_medio_ano}</td>
            <td style='text-align: center'>{ticket_medio_ano}</td>
            <td style='text-align: center'><font color = '{cor(meta_ticket_medio_ano, ticket_medio_ano)}'>◙</font></td>
          </tr>

        </table>



        <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

            <p>Qualquer dúvida estou à disposição.</p>
            <p>Att.,Victor</p>

        """

        #adicionando um anexo
        caminho = Path.cwd() / nome_backup / loja / f'{dia_indicador.month}_{dia_indicador.day} {loja}.xlsx'
        with open(caminho, 'rb') as  arquivo:
            anexo = MIMEBase('apllication', 'octet-stream') # 'apllication', 'octet-stream' é usado para dados binários arbitrários. Salvar o dado
            anexo.set_payload(arquivo.read()) #adicionando o arquivo binário
            encode_base64(anexo) #codificando o arquivo na base 64, ASCII
            anexo.add_header('Content-Disposition', 'attachment', filename = f'{dia_indicador.month}_{dia_indicador.day} {loja}.xlsx') #criando o cabeçalho que vai ser adicionando à parte ao anexo
            mensagem.attach(anexo) #adicionando no corpo do e-mail



        mensagem.attach(MIMEText(texto, 'html'))

        with smtplib.SMTP(host, port) as servidor:
            servidor.starttls() #protocolo de segurança
            servidor.login(usuario, senha)
            servidor.sendmail(mensagem['From'], mensagem['To'], mensagem.as_string().encode('utf-8'))




### Passo 7 - Criar ranking para diretoria

In [ ]:
#criando e ordenando o faturamento anual
faturamento_ano = juntos_df.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_ano.rename(columns={'Valor Final': 'Faturamento'}, inplace=True)
faturamento_ano.sort_values('Faturamento', ascending=False, inplace=True)
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day} Ranking_anual.xlsx'
faturamento_ano.to_excel(nome_backup / nome_arquivo)

#criando e ordenando o faturamento do dia
vendas_dia = juntos_df.loc[juntos_df['Data']==dia_indicador]
faturamento_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_dia.rename(columns={'Valor Final': 'Faturamento'}, inplace=True)
faturamento_dia.sort_values('Faturamento', ascending=False, inplace=True)
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day} Ranking_dia.xlsx'
faturamento_dia.to_excel(nome_backup / nome_arquivo)


### Passo 8 - Enviar e-mail para diretoria

In [ ]:
mensagem = MIMEMultipart() #criando o e-mail
mensagem['From'] = usuario
mensagem['To'] = email_df.loc[email_df['Gerente']=='Diretoria', 'E-mail'].values[0]
mensagem['Subject'] = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
texto = f'''
Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_dia.index[0]} com Faturamento R${faturamento_dia.iloc[0, 0]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_dia.index[-1]} com Faturamento R${faturamento_dia.iloc[-1, 0]:.2f}

Melhor loja do Ano em Faturamento: Loja {faturamento_ano.index[0]} com Faturamento R${faturamento_ano.iloc[0, 0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_ano.index[-1]} com Faturamento R${faturamento_ano.iloc[-1, 0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
Victor
'''
mensagem.attach(MIMEText(texto, 'plain'))

# 3 - adicionando o arquivo como anexo
caminho = Path.cwd() / nome_backup
for relatorio in caminho.iterdir():
    if 'Ranking' in relatorio.name:
        with open(relatorio, 'rb') as arquivo: #abrindo o arquivo no modo de leitura e binário
            anexo = MIMEBase('apllication', 'octet-stream') # 'apllication', 'octet-stream' é usado para dados binários arbitrários. Salvar o dado
            anexo.set_payload(arquivo.read()) #adicionando o arquivo binário
            encode_base64(anexo) #codificando o arquivo na base 64, ASCII
            anexo.add_header('Content-Disposition', 'attachment', filename= relatorio.name) #criando o cabeçalho que vai ser adicionando à parte ao anexo
            mensagem.attach(anexo) #adicionando no corpo do e-mail



# 4 - Criando o servidor e enviando o email


with smtplib.SMTP(host, port) as servidor:
    servidor.starttls()
    servidor.login(usuario, senha)
    servidor.sendmail(mensagem['From'], mensagem['To'], mensagem.as_string().encode('utf-8'))